# Basic Application
讲述一些简单的量子实现
    

## 1.高低位判断
这其实并不属于算法或者应用的某一个，反而是对量子计算编程工具的一个探究：通过固定的量子比特，来探究在编程中量子比特与输出的对应位置关系。

In [15]:
from pyqpanda3.core import *
#创建一个带有四个比特的量子电路
circuit = QCircuit(4)

circuit << X(0)

prog = QProg(circuit)
# 测量所有量子比特,测第i位为第i位置
prog << measure(0,0) << measure(1,1) << measure(2,2) << measure(3,3)



QVM = CPUQVM()

QVM.run(prog,1024)

print(QVM.result().get_prob_dict())
data = QVM.result().get_prob_dict()

# 找到值为1.0的键，并判断'1'的位置
for key, value in data.items():
    if value == 1.0:
        if key[0] == '1':
            print(f"键 '{key}' 中 '1' 在第一位，它被默认为是第0位")
        elif key[-1] == '1':
            print(f"键 '{key}' 中 '1' 在最后一位，它被默认为是第0位")
        break


{'0001': 1.0}
键 '0001' 中 '1' 在最后一位，它被默认为是第0位


## 2.量子隐形传态
量子隐形传态利用叠加态来将一个量子比特的状态传递给另一个量子比特，这并不违背量子不可克隆原理，量子的状态只不过从一个转移到了另一个。

In [173]:
from pyqpanda3.core import *
from pyqpanda3.quantum_info import StateVector

#我们规定Alice有 qubits[0] 和 qubits[1]，Bob有 qubits[2]。其中1和2是我们所制备的纠缠对，目标是将任意量子态0传输到量子比特2上。

#circuitEPR用于制备贝尔态(EPR对)
circuitEPR = QCircuit()
#从计算基出发，第一步是将量子比特1和2纠缠起来，我们在这里以贝尔态为例。
#从计算基制备贝尔态
circuitEPR << H(1) << CNOT(1,2)

#circuitUnknow用于模拟未知量子态
circuitUnknow = QCircuit()
circuitUnknow << H(0)

#circuitDecode用于解码Bell态到计算基上
circuitDecode = QCircuit()
circuitDecode << CNOT(0,1) << H(0)

prog = QProg()
##测量，用于判断恢复操作
prog << circuitEPR << circuitUnknow << circuitDecode << measure(0,0) << measure(1,1)
#根据测量结果执行对应恢复操作

branch_prog00 = QProg()
branch_prog00 << I(2)

branch_prog01 = QProg()
branch_prog01 << Z(2)

branch_prog10 = QProg()
branch_prog10 << X(2)

branch_prog11 = QProg()
branch_prog11 << Z(2) << X(2)

prog << (
    qif([0,1]).then(branch_prog11)
        .qelseif([0]).then(branch_prog01)
        .qelseif([1]).then(branch_prog10)
        .qelse(branch_prog00)

)

print(prog)
QVM = CPUQVM()
QVM.run(prog,1)
print("演化后密度矩阵为",QVM.result().get_state_vector())
stv = StateVector(QVM.result().get_state_vector())
print("对应纯度为",stv.purity())

prog2 = QProg(3)
prog2 << circuitUnknow
QVM2 = CPUQVM()
QVM2.run(prog2,1)
print("初始密度矩阵为",QVM2.result().get_state_vector())
stv2 = StateVector(QVM2.result().get_state_vector())
print("对应纯度为",stv2.purity())







          ┌─┐               ┌─┐     ┌─┐           
q_0:  |0>─┤H├ ────── ───*── ┤H├─── ─┤M├── ─── ─── 
          ├─┤        ┌──┴─┐ └┬┴┐    └╥┘           
q_1:  |0>─┤H├ ───*── ┤CNOT├ ─┤M├── ──╫─── ─── ─── 
          └─┘ ┌──┴─┐ ├─┬──┘  └╥┼─┐   ║┌─┐ ┌─┐ ┌─┐ 
q_2:  |0>──── ┤CNOT├ ┤Z├─── ──╫┤X├ ──╫┤Z├ ┤X├ ┤I├ 
              └────┘ └─┘      ║└─┘   ║└─┘ └─┘ └─┘ 
 c :   / ═════════════════════╩══════╩═
                               1      0


演化后密度矩阵为 [(-0-0j), (-0-0j), (-0-0j), (-0.7071067811865481-0j), 0j, 0j, 0j, (-0.7071067811865481+0j)]
对应纯度为 (0.9999999999999996+0j)
初始密度矩阵为 [(0.7071067811865476+0j), (0.7071067811865476+0j), 0j, 0j, 0j, 0j, 0j, 0j]
对应纯度为 (1.0000000000000004+0j)


## 3.超密编码
超密编码，原理与量子隐形传态类似，都是对叠加态纠缠的运用。它使用泡利矩阵操作来对应编码，从而传递经典信息，实现使用一个量子比特传输两个经典比特。

In [ ]:
from pyqpanda3.core import *

#先制备Bell态
#circuitEPR用于制备贝尔态(EPR对)
circuitEPR = QCircuit()
#从计算基出发，第一步是将量子比特0和1纠缠起来，以贝尔态为例。
#从计算基制备贝尔态
circuitEPR << H(0) << CNOT(0,1)

circuitO = QCircuit()
#对量子比特0进行操作，根据经典比特的状态进行编码.00对应I门，01对应X门，10对应Z门，11对应XZ门，以X门为例
circuitO << X(0)

circuitDecode = QCircuit()
#对量子比特1进行操作，根据经典比特的状态进行解码.从Bell态解码成计算基，解码结果即为传输的经典比特
circuitDecode  << CNOT(0,1) << H(0)

prog = QProg() << circuitEPR << circuitO << circuitDecode

QVM = CPUQVM()
QVM.run(prog,1024)
#结果偏差是阅读顺序的问题。
print(QVM.result().get_prob_dict())


{'00': 0.0, '01': 0.0, '10': 1.0000000000000004, '11': 0.0}
